# Extração da Drogaria São João

In [3]:
# Import do código
import requests
from bs4 import BeautifulSoup
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import json
import pandas as pd
import math

/tmp/ipykernel_11854/3822156316.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# Função de acessar site
all_workers = os.cpu_count()

def access_site(url, max_attempts=all_workers):
    headers = {'Accept': '*/*', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    for retry in range(max_attempts):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response
        except Exception as e:
                print(f"An error occurred while accessing URL: {url}: {e}. Retrying... (Attempt {retry + 1}/{max_attempts})")
                time.sleep(1)

In [82]:
# Capturando todos os produtos
listaProdutos = []
urlRobotsTxt = [
                "https://www.saojoaofarmacias.com.br/sitemap/product-0.xml",
                "https://www.saojoaofarmacias.com.br/sitemap/product-1.xml",
                "https://www.saojoaofarmacias.com.br/sitemap/product-2.xml",
                "https://www.saojoaofarmacias.com.br/sitemap/product-3.xml"
               ]

for j in urlRobotsTxt:
    response = access_site("https://www.saojoaofarmacias.com.br/sitemap/product-0.xml")
    soup = BeautifulSoup(response.text, 'html.parser')
    produtos = soup.find_all('loc')

    for i in produtos:
        produtoCerto = (str(i)).split("<loc>")[1]
        produtoSuperCerto = produtoCerto.split("</loc>")[0]
        listaProdutos.append(produtoSuperCerto)

In [142]:
# Captura do departamento na página individual
listaProdutosCerta = []

for i in listaProdutos:
    response = access_site(i)
    if response is None:
        continue
    
    soup = BeautifulSoup(response.text, 'html.parser')
    departamento = soup.find('a', class_='vtex-breadcrumb-1-x-link vtex-breadcrumb-1-x-link--1 dib pv1 link ph2 c-muted-2 hover-c-link')
    if departamento is not None:
        if departamento.text == 'Medicamentos':
            listaProdutosCerta.append(i)

listaProdutosCerta

In [118]:
# Função que captura do departamento na página individual
def getMedicamentos():
    listaProdutosCerta = []

    for i in listaProdutos:
        response = access_site(i)
        if response is None:
            continue
        
        soup = BeautifulSoup(response.text, 'html.parser')
        departamento = soup.find('a', class_='vtex-breadcrumb-1-x-link vtex-breadcrumb-1-x-link--1 dib pv1 link ph2 c-muted-2 hover-c-link')
        if departamento is not None:
            if departamento.text == 'Medicamentos':
                listaProdutosCerta.append(i)

    return listaProdutosCerta

In [115]:
listaTeste = listaProdutosCerta

In [147]:
# Função de execução rápida
all_workers = os.cpu_count()

listaUrlsMedicamentos = []

with ThreadPoolExecutor(max_workers=all_workers) as executor:
    # Submeter as tarefas e obter os futuros
    futures = [executor.submit(getMedicamentos) for _ in range(1)]
    
    # Processar os resultados à medida que forem completados
    for future in as_completed(futures):
        result = future.result()  # Obter o resultado do futuro
        if isinstance(result, list):  # Garantir que o resultado seja uma lista
            listaUrlsMedicamentos.extend(result)  # Adicionar todos os itens da lista à listaJSON
        else:
            raise ValueError("Esperado uma lista de strings, mas obteve um tipo diferente")

In [136]:
listaTeste

['https://www.saojoaofarmacias.com.br/a-saude-da-mulher-150ml-sem-56/p',
 'https://www.saojoaofarmacias.com.br/actifedrin-c--20-comprimidos-farmoquimica-68/p',
 'https://www.saojoaofarmacias.com.br/aerolin-200-doses-spray-gsk-88/p',
 'https://www.saojoaofarmacias.com.br/albocresil-gel-50g-c--aplicadores-takeda-119/p',
 'https://www.saojoaofarmacias.com.br/albocresil-12ml-solucao-hypera--120/p',
 'https://www.saojoaofarmacias.com.br/amplictil-25mg-c-20-comprimidos-aventis-194/p',
 'https://www.saojoaofarmacias.com.br/amplictil-100mg-c--20-comprimidos-aventis--c1--195/p',
 'https://www.saojoaofarmacias.com.br/amplictil-20ml-aventis--c1--196/p',
 'https://www.saojoaofarmacias.com.br/anafranil-25mg-20-comprimidos-revestidos-sandoz--c1--208/p',
 'https://www.saojoaofarmacias.com.br/atorvastatina-10mg-30-comprimidos-revestidos-ems-genericos-10100896/p',
 'https://www.saojoaofarmacias.com.br/atorvastatina-20mg-30-comprimidos-revestidos-ems-genericos-popular-10100922/p',
 'https://www.saojoaof

In [146]:
# Capturando dados de cada produto
eans = []
precoComDescontos = []
precoSemDescontos = []
descontos = []
marcas = []
nomes = []

for i in listaTeste:
    response = access_site(i)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Nome
    nome = soup.find('span', class_='vtex-store-components-3-x-productBrand vtex-store-components-3-x-productBrand--quickview').text
    nomes.append(nome)

    # Marca
    divMarca = soup.find('div', class_="vtex-store-components-3-x-productBrandContainer")
    marca = divMarca.find('span').text
    marcas.append(marca)

    # EAN
    divEan = soup.find('span', class_='vtex-flex-layout-0-x-flexColChild vtex-flex-layout-0-x-flexColChild--productSpecificationValue pb0')
    if divEan is not None:
        ean = divEan.find('span').text
        eans.append(ean)
    else:
        eans.append("NaN")

    # Preco sem desconto
    precoSemDesconto = soup.find('span', class_='sjdigital-custom-apps-1-x-currencyContainer')
    parcela1 = precoSemDesconto.find('span', class_='sjdigital-custom-apps-1-x-currencyInteger').text
    parcela2 = precoSemDesconto.find('span', class_='sjdigital-custom-apps-1-x-currencyFraction').text
    precoSemDesconto = parcela1 + '.' + parcela2
    precoSemDescontos.append(precoSemDesconto)

    # Preco com desconto
    divPrecoComDesconto = soup.find('span', class_='sjdigital-custom-apps-1-x-currencyContainer')
    if divEan is not None:
        parcela1 = divPrecoComDesconto.find('span', class_='sjdigital-custom-apps-1-x-currencyInteger').text
        parcela2 = divPrecoComDesconto.find('span', class_='sjdigital-custom-apps-1-x-currencyFraction').text
        precoComDesconto = parcela1 + '.' + parcela2
        precoComDescontos.append(precoComDesconto)
    else:
        precoComDesconto = precoSemDesconto
        precoComDescontos.append(precoComDesconto)

    # Desconto
    desconto = (1 - ( float(precoComDesconto) / float(precoSemDesconto)) ) 
    descontos.append(desconto)

KeyboardInterrupt: 

In [138]:
# Criando dataFrame
data = {"nome": nomes, "marcas": marcas, "EAN": eans, "Preço com desconto": precoComDescontos, "Preço sem desconto": precoSemDescontos, "Desconto": descontos}
df = pd.DataFrame(data)

df

,nome,marcas,EAN,Preço com desconto,Preço sem desconto,Desconto
0,A Saúde da Mulher 150ml SEM,Outras,NaN,19.55,19.55,0.0
1,Actifedrin C/ 20 Comprimidos Farmoquímica,Outras,NaN,15.28,15.28,0.0
2,Aerolin 200 Doses Spray GSK,Aerolin,NaN,26.58,26.58,0.0
3,Albocresil Gel 50G C/ Aplicadores Takeda,Outras,NaN,39.04,39.04,0.0
4,Albocresil 12ml solução Hypera,Outras,NaN,39.32,39.32,0.0
5,Amplictil 25Mg C/20 Comprimidos Aventis,Outras,NaN,9.66,9.66,0.0
6,Amplictil 100Mg C/ 20 Comprimidos Aventis (C1),Outras,NaN,14.42,14.42,0.0
7,Amplictil 20Ml Aventis (C1),Outras,NaN,11.71,11.71,0.0
8,Anafranil 25Mg 20 Comprimidos Revestidos Sando...,Outras,NaN,59.11,59.11,0.0
9,Atorvastatina 10Mg 30 Comprimidos Revestidos E...,Outras,NaN,82.14,82.14,0.0


In [81]:
# Exportando tudo para um excel
df.to_excel('DadosSaoJoao.xlsx',sheet_name='Sheet1')